### 1512. Number of Good Pairs



Given an array of integers `nums`.

A pair `(i,j)` is called *good* if `nums[i]` == `nums[j]` and `i` < `j`.

Return the number of *good* pairs.

#### Example 1:

Input: nums = \[1,2,3,1,1,3\]
Output: 4
Explanation: There are 4 good pairs (0,3), (0,4), (3,4), (2,5) 0-indexed.

#### Example 2:

Input: nums = \[1,1,1,1\]
Output: 6
Explanation: Each pair in the array are *good*.

#### Example 3:

Input: nums = \[1,2,3\]
Output: 0

#### Constraints:

-   `1 <= nums.length <= 100`
-   `1 <= nums[i] <= 100`

In [3]:
from typing import List
class Solution:
    def numIdenticalPairs(self, nums: List[int]) -> int:
        c = collections.Counter(nums)
        res = 0
        for key in c:
            if c[key] < 2:
                continue
            res += math.comb(c[key], 2)
        return res

### 1513. Number of Substrings With Only 1s



Given a binary string `s` (a string consisting only of '0' and '1's).

Return the number of substrings with all characters 1's.

Since the answer may be too large, return it modulo 10^9 + 7.

#### Example 1:

Input: s = "0110111"
Output: 9
Explanation: There are 9 substring in total with only 1's characters.
"1" -> 5 times.
"11" -> 3 times.
"111" -> 1 time.

#### Example 2:

Input: s = "101"
Output: 2
Explanation: Substring "1" is shown 2 times in s.

#### Example 3:

Input: s = "111111"
Output: 21
Explanation: Each substring contains only 1's characters.

#### Example 4:

Input: s = "000"
Output: 0

#### Constraints:

-   `s[i] == '0'` or `s[i] == '1'`
-   `1 <= s.length <= 10^5`

In [7]:
class Solution2:
    def numSub(self, s: str) -> int:
        res, temp, mod = 0, 0, 10**9+7
        for c in s:
            if c == '1':
                temp += 1
            elif temp!=0:
                res += temp*(temp+1)//2
                temp = 0
        if temp!=0:
            res += temp*(temp+1)//2
        return res%mod
        '''
        # one line
        return sum(len(a) * (len(a) + 1) // 2 for a in s.split('0')) % (10**9 + 7)
        '''

### 1514. Path with Maximum Probability


You are given an undirected weighted graph of `n` nodes (0-indexed), represented by an edge list where `edges[i] = [a, b]` is an undirected edge connecting the nodes `a` and `b` with a probability of success of traversing that edge `succProb[i]`.

Given two nodes `start` and `end`, find the path with the maximum probability of success to go from `start` to `end` and return its success probability.

If there is no path from `start` to `end`, return 0. Your answer will be accepted if it differs from the correct answer by at most 1e-5.

#### Example 1:

![](https://assets.leetcode.com/uploads/2019/09/20/1558_ex1.png)

Input: n = 3, edges = \[\[0,1\],\[1,2\],\[0,2\]\], succProb = \[0.5,0.5,0.2\], start = 0, end = 2
Output: 0.25000
Explanation: There are two paths from start to end, one having a probability of success = 0.2 and the other has 0.5 \* 0.5 = 0.25.

#### Example 2:

![](https://assets.leetcode.com/uploads/2019/09/20/1558_ex2.png)

Input: n = 3, edges = \[\[0,1\],\[1,2\],\[0,2\]\], succProb = \[0.5,0.5,0.3\], start = 0, end = 2
Output: 0.30000

#### Example 3:

![](https://assets.leetcode.com/uploads/2019/09/20/1558_ex3.png)

Input: n = 3, edges = \[\[0,1\]\], succProb = \[0.5\], start = 0, end = 2
Output: 0.00000
Explanation: There is no path between 0 and 2.

#### . Constraints:

-   `2 <= n <= 10^4`
-   `0 <= start, end < n`
-   `start != end`
-   `0 <= a, b < n`
-   `a != b`
-   `0 <= succProb.length == edges.length <= 2*10^4`
-   `0 <= succProb[i] <= 1`
-   There is at most one edge between every two nodes.

#### Solution with BFS:

In [8]:
from typing import List
class Solution3a:
    def maxProbability(self, n: int, edges: List[List[int]], succProb: List[float], start: int, end: int) -> float:
        def makeGraph():
            graph, nodes = collections.defaultdict(list), set()
            for i in range(len(edges)):
                x, y = edges[i][0], edges[i][1]
                graph[x] += [(y, succProb[i])]
                graph[y] += [(x, succProb[i])]
                nodes |= {x, y}
            return graph, nodes
        graph, nodes = makeGraph()
        if end not in nodes: return 0.0
        # BFS
        probs = [0.0 if i!=start else 1.0 for i in range(n)]
        level = collections.deque([start])
        while level:
            node = level.popleft()
            for v, p in graph[node]:
                # we only consider the path with greater opportunity (a greedy approach)
                # DFS with probability multiply doesn't work, it won't warranty that you reach 
                # a certain point using the minimum number of steps so at some point if the input 
                # is too big the solution will underflow or you will get a TLE
                if probs[node]*p > probs[v]:
                    level.append(v)
                    probs[v] = probs[node]*p
        return probs[end]

#### Solution with Dijkstra:

In [9]:
class Solution:
    def maxProbability(self, n: int, edges: List[List[int]], succProb: List[float], start: int, end: int) -> float:
        def makeGraph():
            graph, nodes = collections.defaultdict(list), set()
            for i in range(len(edges)):
                x, y = edges[i][0], edges[i][1]
                graph[x] += [(y, succProb[i])]
                graph[y] += [(x, succProb[i])]
                nodes |= {x, y}
            return graph, nodes
        graph, nodes = makeGraph()
        if end not in nodes: return 0.0
        '''
        # BFS
        probs = [0.0 if i!=start else 1.0 for i in range(n)]
        level = collections.deque([start])
        while level:
            node = level.popleft()
            for v, p in graph[node]:
                # we only consider the path with greater opportunity (a greedy approach)
                # DFS with probability multiply doesn't work, it won't warranty that you reach 
                # a certain point using the minimum number of steps so at some point if the input 
                # is too big the solution will underflow or you will get a TLE
                if probs[node]*p > probs[v]:
                    level.append(v)
                    probs[v] = probs[node]*p
        return probs[end]
        '''
        # Dijkstra's algorithm
        level = [(-1, start)]
        visited = set()
        while level:
            p, node = heapq.heappop(level)
            if node == end: return -p
            visited.add(node)
            for neighbor, pr in graph[node]:
                if neighbor in visited:
                    continue
                heapq.heappush(level, (p*pr, neighbor))
        return 0.0